In [1]:
from IPython.display import display, Javascript
import json
from numpy.random import uniform, seed
from numpy import floor
from collections import namedtuple

def _tickets_sold(p, demand_level, max_qty):
        quantity_demanded = floor(max(0, p - demand_level))
        return min(quantity_demanded, max_qty)

def simulate_revenue(days_left, tickets_left, pricing_function, rev_to_date=0, 
                     demand_level_min=100, demand_level_max=200, verbose=False):
    if (days_left == 0) or (tickets_left == 0):
        if verbose:
            if (days_left == 0):
                print("The flight took off today. ")
            if (tickets_left == 0):
                print("This flight is booked full.")
            print("Total Revenue: ${:.0f}".format(rev_to_date))
        return rev_to_date
    else:
        demand_level = uniform(demand_level_min, demand_level_max)
        p = pricing_function(days_left, tickets_left, demand_level)
        q = _tickets_sold(demand_level, p, tickets_left)
        if verbose:
            print("{:.0f} days before flight: "
                  "Started with {:.0f} seats. "
                  "Demand level: {:.0f}. "
                  "Price set to ${:.0f}. "
                  "Sold {:.0f} tickets. "
                  "Daily revenue is {:.0f}. Total revenue-to-date is {:.0f}. "
                  "{:.0f} seats remaining".format(days_left, tickets_left, demand_level, p, q, 
                                                  p*q, p*q+rev_to_date, tickets_left-q))
        return simulate_revenue(days_left = days_left-1,
                              tickets_left = tickets_left-q,
                              pricing_function=pricing_function,
                              rev_to_date=rev_to_date + p * q,
                              demand_level_min=demand_level_min,
                              demand_level_max=demand_level_max,
                              verbose=verbose)

def _save_score(score):
    message = {
        'jupyterEvent': 'custom.exercise_interaction',
        'data': {
            'learnTutorialId': 0,
            'interactionType': "check",
            'questionId': 'Aug31OptimizationChallenge',
            'outcomeType': 'Pass',
            'valueTowardsCompletion': score/10000,
            'failureMessage': None,
            'learnToolsVersion': "Testing"
        }
    }
    js = 'parent.postMessage(%s, "*")' % json.dumps(message)
    display(Javascript(js))

def score_me(pricing_function, sims_per_scenario=200):
    seed(0)
    Scenario = namedtuple('Scenario', 'n_days n_tickets')
    scenarios = [Scenario(n_days=100, n_tickets=100),
                 Scenario(n_days=14, n_tickets=50),
                 Scenario(n_days=2, n_tickets=20),
                Scenario(n_days=1, n_tickets=3),
                 ]
    scenario_scores = []
    for s in scenarios:
        scenario_score = sum(simulate_revenue(s.n_days, s.n_tickets, pricing_function)
                                     for _ in range(sims_per_scenario)) / sims_per_scenario
        print("Ran {:.0f} flights starting {:.0f} days before flight with {:.0f} tickets. "
              "Average revenue: ${:.0f}".format(sims_per_scenario,
                                                s.n_days,
                                                s.n_tickets,
                                                scenario_score))
        scenario_scores.append(scenario_score)
    score = sum(scenario_scores) / len(scenario_scores)
    try:
        _save_score(score)
    except:
        pass
    print("Average revenue across all flights is ${:.0f}".format(score))

In [2]:
import numpy as np
from scipy.optimize import brentq, fmin_cobyla

In [3]:
def opt_value(q_bar, A):
    if q_bar <= 0:
        return np.zeros_like(A[:, :1])

    y = 0.5 * A / q_bar
    if y.ndim < 2:
        y = y.reshape(1, -1)

    gfun = lambda C, z: 1 - np.sum(np.maximum(z - C, 0))
    C_opt = np.array([[brentq(gfun, 0, max(max(x), 0), args=(x,))
                       if gfun(0, x) < 0 else 0 for x in y]]).T

    x_opt = np.maximum(y - C_opt, 0)
    V_opt = C_opt + 0.5 * np.sum(x_opt**2, axis=-1, keepdims=True)

    return 2 * q_bar * q_bar * V_opt

In [4]:
class BasePricePolicy(object):
    def __init__(self):
        pass

    def __call__(self, days_left, tickets_left, demand_level):
        return demand_level - 10

In [5]:
class ApproxPricePolicy(BasePricePolicy):
    def __init__(self, demand_bins):
        super(ApproxPricePolicy, self).__init__()
        self.demand_bins = demand_bins

    def __call__(self, days_left, tickets_left, demand_level):
        tickets_left, days_left = int(tickets_left), int(days_left)
        self.compute_dp(days_left, tickets_left)

        demand_level_bin = np.digitize(demand_level, self.demand_bins) - 1

        qty = np.argmax(self.current_[demand_level_bin, :tickets_left + 1]
                        + self.value_[days_left - 1, tickets_left::-1])

        price = demand_level - qty
        return price

    def compute_dp(self, days_left, tickets_left):
        dp_computed_, tickets_left = hasattr(self, "value_"), int(tickets_left)
        if dp_computed_:
            n_days, n_tickets_p1 = self.value_.shape
            dp_computed_ = (n_days >= days_left) and (n_tickets_p1 > tickets_left)
        if dp_computed_:
            return
        
        self.value_, self.current_ = self._compute_dp(days_left, tickets_left)
    
    def _compute_dp(self, n_days, n_tickets):
        current = np.zeros((len(self.demand_bins), 1 + n_tickets), dtype=float)
        for q in range(1 + n_tickets):
            current[:, q] = (self.demand_bins - q) * q
            
        V_tilde = np.zeros((n_days, 1 + n_tickets), dtype=float)
        for t in range(1, n_days):
            for x in range(1 + n_tickets):
                V_txq = current[:, :x + 1] + V_tilde[t - 1, np.newaxis, x::-1]
                V_tilde[t, x] = np.mean(np.max(V_txq, axis=-1), axis=0)
        return V_tilde, current

In [6]:
pricing_function = ApproxPricePolicy(np.linspace(100, 200, num=2001))

In [7]:
simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function, verbose=True)

7 days before flight: Started with 50 seats. Demand level: 106. Price set to $106. Sold 0 tickets. Daily revenue is 0. Total revenue-to-date is 0. 50 seats remaining
6 days before flight: Started with 50 seats. Demand level: 196. Price set to $172. Sold 24 tickets. Daily revenue is 4122. Total revenue-to-date is 4122. 26 seats remaining
5 days before flight: Started with 26 seats. Demand level: 181. Price set to $168. Sold 13 tickets. Daily revenue is 2183. Total revenue-to-date is 6305. 13 seats remaining
4 days before flight: Started with 13 seats. Demand level: 175. Price set to $167. Sold 8 tickets. Daily revenue is 1337. Total revenue-to-date is 7642. 5 seats remaining
3 days before flight: Started with 5 seats. Demand level: 192. Price set to $187. Sold 5 tickets. Daily revenue is 935. Total revenue-to-date is 8578. 0 seats remaining
This flight is booked full.
Total Revenue: $8578


8577.624619799693

In [8]:
score_me(pricing_function)

Ran 200 flights starting 100 days before flight with 100 tickets. Average revenue: $18519
Ran 200 flights starting 14 days before flight with 50 tickets. Average revenue: $8566
Ran 200 flights starting 2 days before flight with 20 tickets. Average revenue: $2923
Ran 200 flights starting 1 days before flight with 3 tickets. Average revenue: $434


<IPython.core.display.Javascript object>

Average revenue across all flights is $7611
